In [1]:
import random
import copy
from collections import defaultdict

In [2]:
class Node:
    def __init__(self, node_id):
        self.id = node_id
        self.connections = []
        
    def __str__(self):
        return f'Node: id – {self.id}, connections – {[connection.get_simple_string() for connection in self.connections]}'

class Connection:
    def __init__(self, connection_id, sender_node_id, receiver_node_id):
        self.id = connection_id
        self.sender_node_id = sender_node_id
        self.receiver_node_id = receiver_node_id
        self.channel_id = self.sender_node_id
        
    def get_simple_string(self):
        return f'({self.sender_node_id}-{self.receiver_node_id})'
        
    def __str__(self):
        return f'Connection: id – {self.id}, sender_node_id-receiver_node_id – {get_simple_string()}'

class Group:
    def __init__(self, group_id):
        self.id = group_id
        self.node_ids = set()
        self.connection_count_by_external_node_ids = defaultdict(int)
        self.endpoint_count_by_channel_id = defaultdict(int)
        
    def __str__(self):
        return f'Group: id – {self.id}, node_ids – {self.node_ids}'


In [7]:
connections_by_id = {}
nodes_by_id = {}

with open('circuit.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(';')
        
        try:
            node_id = int(parts[1])
        except ValueError:
            continue
        
        nodes_by_id[node_id] = Node(node_id)

with open('circuit.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(';')
        
        try:
            node_id = int(parts[1])
        except ValueError:
            continue
            
        node_id_strings_list = parts[2].split(',')
        for node_id_string in node_id_strings_list:
            try:
                sender_node_id = int(node_id_string)
            except ValueError:
                continue
            
            connections_by_id[len(connections_by_id)] = Connection(len(connections_by_id), sender_node_id, node_id)
            
for connection in connections_by_id.values():
    nodes_by_id[connection.sender_node_id].connections.append(connection)
    nodes_by_id[connection.receiver_node_id].connections.append(connection)
            

connection_count_by_node_id = defaultdict(int)
node_id_by_connection_count = defaultdict(set)

for connection in connections_by_id.values():
    connection_count_by_node_id[connection.sender_node_id] += 1
    connection_count_by_node_id[connection.receiver_node_id] += 1

for node_id, connection_count in connection_count_by_node_id.items():
    node_id_by_connection_count[connection_count].add(node_id)

connection_count_by_node_id = dict(connection_count_by_node_id)    
node_id_by_connection_count = dict(node_id_by_connection_count)

connected_node_ids_by_node_id = defaultdict(set)

for connection in connections_by_id.values():
    connected_node_ids_by_node_id[connection.sender_node_id].add(connection.receiver_node_id)
    connected_node_ids_by_node_id[connection.receiver_node_id].add(connection.sender_node_id)
    
connected_node_ids_by_node_id = dict(connected_node_ids_by_node_id)

endpoint_count_by_channel_id = defaultdict(int)

for connection in connections_by_id.values():
    endpoint_count_by_channel_id[connection.channel_id] += 2
    
endpoint_count_by_channel_id = dict(endpoint_count_by_channel_id)

channel_ids_by_node_id = defaultdict(set)

for connection in connections_by_id.values():
    channel_ids_by_node_id[connection.sender_node_id].add(connection.channel_id)
    channel_ids_by_node_id[connection.receiver_node_id].add(connection.channel_id)
    
channel_ids_by_node_id = dict(channel_ids_by_node_id)

In [8]:
max_nodes_per_group = 8
max_channels_total = 3

In [9]:
groups_by_id = {}
node_ids_all = set(nodes_by_id.keys())
node_ids_claimed = set()
max_connection_count = max(node_id_by_connection_count)

def get_random_id_from_set(id_set):
    return random.choice(
        tuple(
            id_set
        )
    )

def get_unclaimed_min_connection_nodes():
    node_ids = set()
    curr_min_num_connections = min(node_id_by_connection_count)
    
    while len(node_ids) <= 0:
        node_ids = node_id_by_connection_count[curr_min_num_connections] - node_ids_claimed
        curr_min_num_connections += 1

    return node_ids

def claim_node_id_for_group(group, node_id):
    node_ids_claimed.add(node_id)
    add_node_id_to_group(group, node_id)

def add_node_id_to_group(group, node_id):
    group.node_ids.add(node_id)
    
    for channel_id in channel_ids_by_node_id[node_id]:
        group.endpoint_count_by_channel_id[channel_id] += 1
        
    for connected_node_id in connected_node_ids_by_node_id[node_id]:
        group.connection_count_by_external_node_ids[connected_node_id] += 1
    
def disclaim_node_id_for_group(group, node_id):
    node_ids_claimed.remove(node_id)
    remove_node_id_from_group(group, node_id)
    
def remove_node_id_from_group(group, node_id):
    group.node_ids.remove(node_id)
    
    for channel_id in channel_ids_by_node_id[node_id]:
        group.endpoint_count_by_channel_id[channel_id] -= 1
    
    removed_channel_ids = []
    for channel_id, endpoint_count in group.endpoint_count_by_channel_id.items():
        if endpoint_count <= 0:
            removed_channel_ids.append(channel_id)
    
    for removed_channel_id in removed_channel_ids:
        group.endpoint_count_by_channel_id.pop(removed_channel_id)
        
    removed_connected_node_ids = []    
    for external_node_id, connection_count in group.connection_count_by_external_node_ids.items():
        if connection_count <= 0:
            removed_connected_node_ids.append(external_node_id)
    
    for removed_connected_node_id in removed_connected_node_ids:
        group.connection_count_by_external_node_ids.pop(removed_connected_node_id)

def get_external_channel_ids_for_group(group):
    external_channel_ids = set()
    
    for channel_id, endpoint_count in group.endpoint_count_by_channel_id.items():
        if endpoint_count_by_channel_id[channel_id] != endpoint_count:
            external_channel_ids.add(channel_id)
    
    return external_channel_ids

def is_valid_group(group):
    return (
        len(group.node_ids) < max_nodes_per_group
        and len(get_external_channel_ids_for_group(group)) <= max_channels_total
    )

while len(node_ids_claimed) < len(node_ids_all):
    group = Group(len(groups_by_id) + 1)
    
    starting_node = get_random_id_from_set(get_unclaimed_min_connection_nodes())
    claim_node_id_for_group(group, starting_node)
    
    candidate_nodes = set(group.connection_count_by_external_node_ids.keys()) - node_ids_claimed

    while len(node_ids_claimed) < len(node_ids_all) and len(candidate_nodes) > 0:
        curr_node_id = candidate_nodes.pop()

        claim_node_id_for_group(group, curr_node_id)

        if is_valid_group(group):
            candidate_nodes.update(connected_node_ids_by_node_id[curr_node_id] - node_ids_claimed)
        else:
            disclaim_node_id_for_group(group, curr_node_id)
        
    groups_by_id[group.id] = group

In [10]:
groups = list(groups_by_id.values())

merged_groups = copy.deepcopy(groups)
random.shuffle(merged_groups)

is_merging = True
curr_group_index = 0

while curr_group_index < len(merged_groups):
    curr_group = merged_groups[curr_group_index]
    is_completed_merge = False
    
    group_indexs = list(range(curr_group_index + 1, len(merged_groups)))
    random.shuffle(group_indexs)
    
    for group_index in group_indexs:
        group_node_ids = list(merged_groups[group_index].node_ids)
        
        for node_id in group_node_ids:
            add_node_id_to_group(curr_group, node_id)
            
        if is_valid_group(curr_group):
            del merged_groups[group_index]
            merged_groups[curr_group_index] = curr_group
            is_completed_merge = True
            break
        
        for node_id in group_node_ids:
            remove_node_id_from_group(curr_group, node_id)
            
    if not is_completed_merge:
        curr_group_index += 1

In [11]:
external_channel_ids_by_group_id = {}
external_channel_ids = set()

for group_id, group in groups_by_id.items():
    external_channel_ids_by_group_id[group_id] = get_external_channel_ids_for_group(group)
    external_channel_ids.update(get_external_channel_ids_for_group(group))

In [12]:
import networkx as nx

def graph_coloring(graph, m):
    colours = {}
    nodes = list(graph.nodes())
    
    nodes = sorted(nodes, key=lambda x: len(list(graph.neighbors(x))), reverse=True)
    
    for node in nodes:
        used_colours = set(colours.get(n, None) for n in graph.neighbors(node))
        available_colours = set(range(1, m+1)) - used_colours
        if available_colours:
            colours[node] = min(available_colours)
        else:
            raise Exception("Graph cannot be colored with %d colors" % m)
    
    return colours

In [13]:
edge_pairs = []

for external_channel_ids in external_channel_ids_by_group_id.values():
    curr_external_channel_ids = list(external_channel_ids)
    
    for i in range(len(curr_external_channel_ids)):
        for j in range(i + 1, len(curr_external_channel_ids)):
            edge_pairs.append((curr_external_channel_ids[i], curr_external_channel_ids[j]))

G = nx.Graph()
G.add_edges_from(edge_pairs)

colors = graph_coloring(G, 5)

In [14]:
for group in groups_by_id.values():
    print(group)

Group: id – 1, node_ids – {98, 105, 92, 94}
Group: id – 2, node_ids – {77, 78, 79, 80, 81}
Group: id – 3, node_ids – {72, 73, 74, 75, 76}
Group: id – 4, node_ids – {101, 102, 103, 113}
Group: id – 5, node_ids – {36, 7, 18}
Group: id – 6, node_ids – {130, 131, 123, 127}
Group: id – 7, node_ids – {64, 63}
Group: id – 8, node_ids – {68, 70, 71}
Group: id – 9, node_ids – {65, 66, 67}
Group: id – 10, node_ids – {56, 59, 61}
Group: id – 11, node_ids – {8, 10, 20}
Group: id – 12, node_ids – {5, 17, 26}
Group: id – 13, node_ids – {6, 16, 25}
Group: id – 14, node_ids – {3, 41, 47}
Group: id – 15, node_ids – {11, 121, 122, 125, 126}
Group: id – 16, node_ids – {91, 69}
Group: id – 17, node_ids – {1, 4, 12}
Group: id – 18, node_ids – {2, 13}
Group: id – 19, node_ids – {119, 120, 124}
Group: id – 20, node_ids – {33, 43}
Group: id – 21, node_ids – {112, 116}
Group: id – 22, node_ids – {128, 129}
Group: id – 23, node_ids – {115, 117}
Group: id – 24, node_ids – {34, 44}
Group: id – 25, node_ids – {40,

In [15]:
colours = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
}

for gate_id, color_id in colors.items():
    colours[color_id].append(gate_id)
    
for gates in colours.values():
    print(sorted(gates))

[1, 2, 3, 11, 18, 20, 24, 25, 26, 39, 44, 50, 52, 61, 62, 76, 85, 86, 90, 100, 105, 107, 108, 111, 129]
[5, 8, 12, 21, 23, 27, 32, 36, 43, 48, 49, 51, 57, 64, 81, 88, 89, 98, 106, 113, 114, 118]
[4, 6, 9, 10, 13, 19, 29, 35, 37, 46, 47, 53, 54, 83, 84, 91, 95, 96, 97, 103, 110, 116, 126]
[14, 22, 28, 31, 45, 60, 67, 82, 87, 99, 109, 117]
[38, 71]


In [74]:
','.join([str(node_id) for node_id in list(group.node_ids)])

'118'